<center>
    <h1>   
         Clinical Features Model to Predict Stroke 
    </h1>
</center>
 
<center><img src="https://image-cdn.medkomtek.com/_A7vRjeLyfZqshjKM71BAmEmmKU=/673x373/smart/klikdokter-media-buckets/medias/2309095/original/050687900_1572589916-Stroke-infarct-By-peterschreiber.media-Shutterstock_1423084877.jpg" alt="Stroke" width="600" ></center>

<center> <h3> Please Upvote if you like the work 😊 </h3></center>

<center>Don't hesitate to give your comment on this notebook. It gives me motivation to improve the analysis in the future</center>

# Introduction
This program is used to find the right model for the Telecomunication dataset. 

# Contents
Contents:

* [1. Load Packages](#1)
* [2. Load Dataset](#2)
* [3. Drop NaN and duplicates values](#3)
* [4. Do Explanatory Data Analysis](#4)
* [5. Mapping Sentiment](#5) 
* [6. Modelling](#6)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>
    
# <font size="+2" color="indigo"><b>1. Load Packages</b></font><br>

In [ ]:
# runtime
import timeit

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# preprocessing
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer

# Ml model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

np.warnings.filterwarnings('ignore')

<a id="2"></a>
    
# <font size="+2" color="indigo"><b>2. Load Dataset</b></font><br>

In [ ]:
data_df=pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
data_df.head()

In [ ]:
data_df.info()

<a id="3"></a>
    
# <font size="+2" color="indigo"><b>3. Find Null and Duplicate Values</b></font><br>

In [ ]:
display(data_df.isnull().sum())
print("====///====")
display(data_df.duplicated().sum())
print("====///====")
display(data_df.info())

> Found null values in bmi columns --> needs to be treated

In [ ]:
data_df=data_df.dropna(axis='rows')


<a id="4"></a>
    
# <font size="+2" color="indigo"><b>4. Exploratory Data Analysis</b></font><br>

## <font size="+1" color="indigo"><b>Affect of Gender on Stroke</b></font><br> 

In [ ]:
# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

ch = {1: 'Having Stroke', 0: 'Not Having Stroke'}
analyze_data=data_df.copy()
analyze_data['stroke'] = data_df['stroke'].map(ch)

gender = analyze_data.groupby(['gender', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()
gender

In [ ]:
fig = px.sunburst(gender, path = ['gender', 'stroke'], values = 'count', color = 'gender', title = 'Affect of Age on Stroke', width = 600, height = 600)
fig.update_traces(textinfo = 'label + percent parent')
fig.show()

## <font size="+1" color="indigo"><b>Distribution of Age of People Having Stroke</b></font><br> 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

value = analyze_data[analyze_data['stroke']=='Having Stroke']['age']

ax.hist(value, bins=10)
plt.title('Distribution of Age of People Having Stroke')

plt.show()

## <font size="+1" color="indigo"><b>Distribution of BMI of People Having Stroke</b></font><br> 

For certain individuals, BMI is a reliable measure of body fatness. It is used to scan for weight ranges that could be associated with health issues.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

value = analyze_data[analyze_data['stroke']=='Having Stroke']['bmi']

ax.hist(value, bins=10)
plt.title('Distribution of BMI of People Having Stroke')

plt.show()

## <font size="+1" color="indigo"><b>Affect of Residence_type on Stroke</b></font><br> 

In [ ]:
Residence_type = analyze_data.groupby(['Residence_type', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()

fig = px.sunburst(Residence_type, path = ['Residence_type', 'stroke'], values = 'count', color = 'Residence_type', title = 'Affect of Residence_type on Stroke', width = 600, height = 600)
fig.update_traces(textinfo = 'label + percent parent')
fig.show()

## <font size="+1" color="indigo"><b>Affect of smoking on Stroke</b></font><br> 

In [ ]:
smoking_status = analyze_data.groupby(['smoking_status', 'stroke']).agg({'stroke': 'count'}).rename(columns = {'stroke': 'count'}).reset_index()

fig = px.sunburst(smoking_status, path = ['smoking_status', 'stroke'], values = 'count', color = 'smoking_status', title = 'Affect of smoking_status on Stroke', width = 600, height = 600)
fig.update_traces(textinfo = 'label + percent parent')
fig.show()

In [ ]:
data_df.head()

In [ ]:
ax = sns.countplot(x="stroke", data=data_df)

> We need to use SMOTE method in order to overcome the problem of unbalanced data

<a id="5"></a>
    
# <font size="+2" color="indigo"><b>5. Label Encoding</b></font><br>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = data_df.drop(['stroke','id'], axis = 1)
y = data_df['stroke']

# Split feature as categorical and continuous features

cat_features=X.select_dtypes(exclude=np.number).columns
cont_features=X.select_dtypes(include=[np.number,'float64','int64']).columns


In [ ]:
# Label Encoder for Categorical Features
def label_encoder(df):
    for i in cat_features:
        le = LabelEncoder()
        df[i] = le.fit_transform(df[i])
    return df

In [ ]:
# Standard Scaler for Continuous Features
sc = StandardScaler()
X[cont_features] = sc.fit_transform(X[cont_features])

# Label encoding for Categorical Features
X = label_encoder(X)

X.head(3)

<a id="6"></a>
    
# <font size="+2" color="indigo"><b>6. Exploratory Data Analysis (Part 2)</b></font><br>

In [ ]:
data_df2=X.join(y)
corrmat = data_df2.corr(method='pearson')
f, ax = plt.subplots(figsize=(24, 16))
sns.heatmap(corrmat, ax=ax, cmap="YlGnBu", linewidths=0.1, annot=True)
data_df2.head()

<a id="7"></a>
    
# <font size="+2" color="indigo"><b>7. Modelling</b></font><br>
## <font size="+1" color="indigo"><b>Data Splitting</b></font><br>

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.15,random_state=0)

## <font size="+1" color="indigo"><b>XGBClassifier</b></font><br>

In [ ]:
Model=XGBClassifier(eval_metric="logloss")
#pipeModel=Pipeline([('scaler', StandardScaler()), ('model', Model)])

# the benefit of using K-Fold is that we could calculate the cross validation value using some of the methods of scoring 
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
cv_results = cross_val_score(Model, X_train, y_train, cv=kfold)

Model.fit(X_train, y_train)
# this is the scaled XGBClassifier
print('Score for XGBClassifier method:', Model.score(X_test, y_test))

# the mean result (10 data) of negative mean squared error
print('Score for XGBClassifier method using cross_val_score:', cv_results.mean())
y_pred = Model.predict(X_test)
y_prob = Model.predict_proba(X_test)[:,1]

#print(mean_squared_error(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("roc_auc score:", roc_auc_score(y_test,y_prob))

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('\n\n-----------------------------------------------------\n\n')

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

print('\n\n-----------------------------------------------------\n\n')
plt.figure(figsize = (8, 8))
from matplotlib import pyplot
from xgboost import plot_importance
plot_importance(Model, max_num_features=10) # top 10 most important features
plt.show()

print('\n\n-----------------------------------------------------\n\n')
arr=confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(arr, range(2), range(2))
plt.figure(figsize = (8, 8))
sns.heatmap(arr, cmap = 'Blues', annot = True, fmt = 'd', annot_kws = {'fontsize': 15},
           yticklabels = ['Stayed', 'Left'], xticklabels = ['Predicted stayed', 'Predicted left'])
plt.show()

## <font size="+1" color="indigo"><b>Logistic Regression</b></font><br>

In [ ]:
Model=LogisticRegression(random_state = 22)
#pipeModel=Pipeline([('scaler', StandardScaler()), ('model', Model)])

# the benefit of using K-Fold is that we could calculate the cross validation value using some of the methods of scoring 
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
cv_results = cross_val_score(Model, X_train, y_train, cv=kfold)

Model.fit(X_train, y_train)

print('Score for Logistic Regression method:', Model.score(X_test, y_test))

# the mean result (10 data) of negative mean squared error
print('Score for Logistic Regression method using cross_val_score:', cv_results.mean())
y_pred = Model.predict(X_test)
y_prob = Model.predict_proba(X_test)[:,1]

#print(mean_squared_error(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("roc_auc score:", roc_auc_score(y_test,y_prob))

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('\n\n-----------------------------------------------------\n\n')

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

print('\n\n-----------------------------------------------------\n\n')
plt.figure(figsize = (8, 8))

feature_importance = abs(Model.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5

featfig = plt.figure()
featax = featfig.add_subplot(1, 1, 1)
featax.barh(pos, feature_importance[sorted_idx], align='center')
featax.set_yticks(pos)
featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
featax.set_xlabel('Relative Feature Importance')

plt.tight_layout()   
plt.show()

print('\n\n-----------------------------------------------------\n\n')
arr=confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(arr, range(2), range(2))
plt.figure(figsize = (8, 8))
sns.heatmap(arr, cmap = 'Blues', annot = True, fmt = 'd', annot_kws = {'fontsize': 15},
           yticklabels = ['Stayed', 'Left'], xticklabels = ['Predicted stayed', 'Predicted left'])
plt.show()

## <font size="+1" color="indigo"><b>Gradient Boosting Classifier</b></font><br>

In [ ]:
Model=GradientBoostingClassifier()
#pipeModel=Pipeline([('scaler', StandardScaler()), ('model', Model)])

# the benefit of using K-Fold is that we could calculate the cross validation value using some of the methods of scoring 
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
cv_results = cross_val_score(Model, X_train, y_train, cv=kfold)

Model.fit(X_train, y_train)

print('Score for GBC method:', Model.score(X_test, y_test))

# the mean result (10 data) of negative mean squared error
print('Score for GBC method using cross_val_score:', cv_results.mean())
y_pred = Model.predict(X_test)
y_prob = Model.predict_proba(X_test)[:,1]

#print(mean_squared_error(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("roc_auc score:", roc_auc_score(y_test,y_prob))

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('\n\n-----------------------------------------------------\n\n')

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

print('\n\n-----------------------------------------------------\n\n')
plt.figure(figsize = (8, 8))

feature_importance = abs(Model.feature_importances_)
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5

featfig = plt.figure()
featax = featfig.add_subplot(1, 1, 1)
featax.barh(pos, feature_importance[sorted_idx], align='center')
featax.set_yticks(pos)
featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
featax.set_xlabel('Relative Feature Importance')

plt.tight_layout()   
plt.show()

print('\n\n-----------------------------------------------------\n\n')
arr=confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(arr, range(2), range(2))
plt.figure(figsize = (8, 8))
sns.heatmap(arr, cmap = 'Blues', annot = True, fmt = 'd', annot_kws = {'fontsize': 15},
           yticklabels = ['Stayed', 'Left'], xticklabels = ['Predicted stayed', 'Predicted left'])
plt.show()


> XGBoost gives best results based on accuracy. The hyperparameter need to be tuned.

<a id="8"></a>
    
# <font size="+2" color="indigo"><b>8. Tuning Hyperparameter</b></font><br>

In [ ]:
#List Hyperparameters yang akan diuji
#penalty = ['l1', 'l2']
#C = np.logspace(-4,4,20)

#Menjadikan ke dalam bentuk dictionary
#hyperparameters = dict(penalty=penalty, C=C)

param_tuning = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7],
    'n_estimators' : [100, 200, 500],
    'objective': ['reg:squarederror']
}

#Membuat Object Logistic Regression
logreg = XGBClassifier(eval_metric="logloss")

#Memasukan ke Grid Search
#CV itu Cross Validation
#Menggunakan 10-Fold CV
clf = GridSearchCV(logreg, param_grid = param_tuning, cv=10)

#Fitting Model
best_model = clf.fit(X,y)

#Nilai hyperparameters terbaik
print('Best Params:', best_model.best_estimator_.get_params())

#Prediksi menggunakan model baru
#y_pred = best_model.predict(x_test)

#Check performa dari model
#print(classification_report(y_test, y_pred))
#roc_auc_score(y_test, y_pred)

In [ ]:
Model=XGBClassifier(eval_metric="logloss", learning_rate=best_model.best_estimator_.get_params()['learning_rate'],
                        max_depth=best_model.best_estimator_.get_params()['max_depth'],
                        min_child_weight=best_model.best_estimator_.get_params()['min_child_weight'],
                        subsample=best_model.best_estimator_.get_params()['subsample'],
                        colsample_bytree=best_model.best_estimator_.get_params()['colsample_bytree'],
                        n_estimators=best_model.best_estimator_.get_params()['n_estimators'],
                        objective=best_model.best_estimator_.get_params()['objective'])
#pipeModel=Pipeline([('scaler', StandardScaler()), ('model', Model)])

# the benefit of using K-Fold is that we could calculate the cross validation value using some of the methods of scoring 
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
cv_results = cross_val_score(Model, X_train, y_train, cv=kfold)

Model.fit(X_train, y_train)

print('Score for XGBoost method:', Model.score(X_test, y_test))

# the mean result (10 data) of negative mean squared error
print('Score for XGBoost method using cross_val_score:', cv_results.mean())
y_pred = Model.predict(X_test)
y_prob = Model.predict_proba(X_test)[:,1]

#print(mean_squared_error(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("roc_auc score:", roc_auc_score(y_test,y_prob))

# Roc curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('\n\n-----------------------------------------------------\n\n')

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

print('\n\n-----------------------------------------------------\n\n')

plt.figure(figsize = (8, 8))
from matplotlib import pyplot
from xgboost import plot_importance
plot_importance(Model, max_num_features=10) # top 10 most important features
plt.show()

print('\n\n-----------------------------------------------------\n\n')
arr=confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(arr, range(2), range(2))
plt.figure(figsize = (8, 8))
sns.heatmap(arr, cmap = 'Blues', annot = True, fmt = 'd', annot_kws = {'fontsize': 15},
           yticklabels = ['Stayed', 'Left'], xticklabels = ['Predicted stayed', 'Predicted left'])
plt.show()